In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm.notebook import tqdm

In [ ]:
torch.cuda.empty_cache()
if torch.cuda.is_available():
    print("GPU available")
    device = "cuda:0"
else:
    print("CPU available")
    device = "cpu"
    
device = torch.device(device)
print(device)




os.path.join --> Path Combination \
os.listdir   --> Converting all fiels in specifed path into list


In [ ]:
data_dir = os.path.join("/kaggle","input","cityscapes-image-pairs","cityscapes_data")
train_dir= os.path.join(data_dir,"train")
val_dir  = os.path.join(data_dir,"val")
train_lst= os.listdir(train_dir)
val_lst  = os.listdir(val_dir)

In [ ]:
print(len(train_lst),len(val_lst))
print(train_lst[0])

In [ ]:
sample_image_os = os.path.join(train_dir, train_lst[0]) 
print(type(sample_image_os)) # string 
print(sample_image_os)# total path


PIL image = Image.open(path)

In [ ]:
sample_image = Image.open(sample_image_os)
plt. figure()
fig = plt. figure()
fig. set_size_inches(50,50)
plt.imshow(sample_image)


 cityscape_np, label_np = splite_image(Combined_Image_np)

In [ ]:
def split_image(image):
    img_np = np.array(image)
    cityscape = img_np[:, :256, :]
    label     = img_np[:, 256: ,:]
    return cityscape, label


PIL = Image.fromarray(np)

In [ ]:
sample_image_np = np.array(sample_image)
print(sample_image_np.shape)
cityscape, label = split_image(sample_image_np)
cityscape_PIL = Image.fromarray(cityscape)
label_PIL = Image.fromarray(label)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(cityscape_PIL)
axes[1].imshow(label_PIL)

In [ ]:
sample_image_torch = torch.tensor(sample_image_np)
print(sample_image_torch.shape)
print(sample_image_np.shape)
plt.imshow(sample_image_torch)

np.random.choice(x,y)\
x --> range of numbers [ 0 : x - 1 ]\
y --> number of requierd generated numbers

In [ ]:
num_items = 1000
color_array = np.random.choice(range(256), 3*num_items).reshape(-1, 3) # 1000 * 3 
print(color_array.shape)
# jusat for clarification
print(color_array[:5, :])
print(color_array.min())
print(color_array.max())

In [ ]:
# clarrification
num_classes = 10
label_model2 = KMeans(n_clusters=num_classes)
label_model2.fit(color_array)# 2D array
print(type(label_model2))

In [ ]:
label_model2.labels_
# 1D array

In [ ]:
len(label_model2.labels_)

In [ ]:
label_model2.predict(color_array[:7])

In [ ]:

cityscape, label = split_image(sample_image)
label_class = label_model.predict(label.reshape(-1, 3)).reshape(256, 256)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cityscape)
axes[1].imshow(label)
axes[2].imshow(label_class)
print(label.shape)
print(label_class.shape)
print(label_class[0][0])
print(label_class[5][150])


# Clustring 

In [ ]:
num_classes = 10
label_model = KMeans(n_clusters=num_classes)
label_model.fit(label.reshape(-1,3))
# 2D array

In [ ]:

cityscape, label = split_image(sample_image)
label_class = label_model.predict(label.reshape(-1, 3)).reshape(256, 256)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cityscape)
axes[1].imshow(label)
axes[2].imshow(label_class)
print(label.shape)
print(label_class.shape)
print(label_class[0][0])
print(label_class[5][150])
label_class2 = torch.tensor(label_class)
print(type(label_class2))
print(label_class[5][150])
print(type(torch.max(label_class2)))
label_class2 = label_class2.long()
print(label_class[5][150])
print(type(torch.max(label_class2)))



# Define DataSet

In [ ]:
class Cityscape_dataset(Dataset):
    def __init__(self, image_dir, label_model):
        self.image_dir = image_dir
        self.image_lst = os.listdir(image_dir)
        self.label_model = label_model
    
    def __len__(self):
        return len(self.image_lst)
    
    def __getitem__(self, idx):
        image_name       = self.image_lst[idx]
        image_path       = os.path.join(self.image_dir, image_name)
        image_colored    = Image.open(image_path).convert('RGB')    
        cityscape, label = split_image(image_colored)
        label_class      = self.label_model.predict(label.reshape(-1,3)).reshape(256,256)
        cityscape        = self.transform(cityscape)
        label_class      = torch.tensor(label_class).long()
        return cityscape, label_class
    
    def split_image(self, img):
        image = np.array(image)
        cityscape, label = image[:, :256, :], image[:, 256:, :]
        return cityscape, label
    
    def transform(self, img):
        Transform = transforms.Compose([
            transforms.ToTensor(),
             transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return Transform(img)

In [ ]:
dataset = Cityscape_dataset(train_dir, label_model)


In [ ]:

class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__() 
        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, 1, 1, bias = False), # bias = False cause we will use BatchNorm next
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_c, out_c, 3, 1, 1, bias = False), # bias = False cause we will use BatchNorm next
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace = True)
        )

        def forward(self, x_img):
            return self.conv(x_img)
        
num_of_classes = 10
class UNET(nn.Module):
    def __init__(self, in_channels=1,out_channels=1,features=[64, 128, 256, 512],):
        super().__init__()
        self.ups  = nn.ModuleList() # Holds submodules in a list
        self.downs= nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    
        # down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
              nn.ConvTranspose2d(
                  feature*2,
                  feature,
                  kernel_size=2,
                  stride =2
              )
          )
            self.ups.append(
              DoubleConv(
                  feature*2,
                  feature
              )
          )
          # End Of Loop

       #bottleneck of UNET
        self.bottleneck = DoubleConv(
            features[-1],
            features[-1]*2
            )
   
  
       #Final Conv (last layer)
        self.final_conv = nn.Conv2d(
            features[0],
            out_channels,
            kernel_size = 1
        )

      # Forward
    def forward(self,x_img):
        #Skip Connection
        skip_connections = []
        # Down
        for down in self.downs:
            x_img = down(x_img)
            skip_connections.append(x_img)
            x_img = self.pool(x_img)

        x_img = self.bottleneck(x_img)
        # reverse skip connection to be from down to up
        skip_connections = skip_connections[::-1]
        # Up
        for idx in range(0, len(self.ups), 2): # Up Then Double Conv
            x_img = self.ups[idx](x_img)
            skip_connection = skip_connections[idx // 2]
            if x_img.shape != skip_connection.shape:
                x_img = TF.resize(x_img, size=skip_connection.shape[2:])
            concat_skip = torch.cat((skip_connection, x_img), dim = 1) 
      #Double Conv
            x_img = self.ups[idx+1](concat_skip)

        #Final Conv, (1*1) convolution
        return self.final_conv(x_img)

In [ ]:
batch_size = 16
epochs = 10
lr = 0.001

In [ ]:
dataset = Cityscape_dataset(train_dir, label_model)
data_loader = DataLoader(dataset, batch_size=batch_size)

model = UNET(in_channels=3, out_channels=10).to(device)
print(model)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
X, Y = iter(data_loader).next()
print(X.shape, Y.shape)
Y_pred = model(X)
print(Y_pred.shape)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

step_losses  = []
epoch_losses = []

for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    for X, Y in tqdm(data_loader, total=len(data_loader), leave=False):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        Y_pred = model(X)
        loss = criterion(Y_pred, Y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        step_losses.append(loss.item())
    print(epoch_loss)
    epoch_losses.append(epoch_loss/len(data_loader))

In [ ]:
# version one
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 2
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 3
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 4
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 5
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 6
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 7
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 8 # after 8 * 30 epoces
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# version 8 # after 8 * 30  + 5 epoces
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
# model_name = "U-Net.pth11"
# torch.save(model.state_dict(), model_name)
model_path = "../input/citysvapev8/U-Net.pth8"


In [ ]:
model_ = UNET(in_channels=3, out_channels=10).to(device)
model_.load_state_dict(torch.load(model_path))

In [ ]:
test_batch_size = 8
dataset = Cityscape_dataset(val_dir, label_model)
data_loader = DataLoader(dataset, batch_size=test_batch_size)

In [ ]:
X, Y = next(iter(data_loader))

In [ ]:
Y_pred = model_(X)

# Testing

In [ ]:

print(Y_pred.shape)
Y_pred = torch.argmax(Y_pred, dim=1)
print(Y_pred.shape)

inverse_transform = transforms.Compose([
    transforms.Normalize((-0.485/0.229, -0.456/0.224, -0.406/0.225), (1/0.229, 1/0.224, 1/0.225))
])

In [ ]:
fig, axes = plt.subplots(test_batch_size, 3, figsize=(3*5, test_batch_size*5))
iou_scores = []

for i in range(test_batch_size):
    
    landscape = inverse_transform(X[i]).permute(1, 2, 0).cpu().detach().numpy()
    label_class = Y[i].cpu().detach().numpy()
    label_class_predicted = Y_pred[i].cpu().detach().numpy()
    
    # IOU score
    intersection = np.logical_and(label_class, label_class_predicted)
    union = np.logical_or(label_class, label_class_predicted)
    iou_score = np.sum(intersection) / np.sum(union)
    iou_scores.append(iou_score)

    axes[i, 0].imshow(landscape)
    axes[i, 0].set_title("Landscape")
    axes[i, 1].imshow(label_class)
    axes[i, 1].set_title("Label Class")
    axes[i, 2].imshow(label_class_predicted)
    axes[i, 2].set_title("Label Class - Predicted")

print("got accurancy = ",sum(iou_scores) / len(iou_scores))
